In [ ]:
from chatbot import Chatbot

In [ ]:
# hyperparameters
model_name = 'NousResearch/Llama-2-7b-hf' # The model to fine-tune, 'NousResearch/Meta-Llama-3-8B', 'NousResearch/Llama-2-7b-hf', 'NousResearch/Llama-2-7b-chat-hf',
prompt_format = 'llama'
device_map = 'cuda:0'
trained_model = ''

In [ ]:
# load the chatbot
chatbot = Chatbot(model_name, trained_model, device=device_map, format=prompt_format)

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, 'float16')

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # Activate 4-bit precision base model loading
    bnb_4bit_quant_type='nf4', # Quantization type (fp4 or nf4)
    bnb_4bit_compute_dtype=compute_dtype, # Compute dtype for 4-bit base models
    bnb_4bit_use_double_quant=False, # Activate nested quantization for 4-bit base models (double quantization)
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='cuda:0',
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

model = PeftModel.from_pretrained(model, trained_model)
model = model.merge_and_unload()

In [ ]:
model = PeftModel.from_pretrained(model, trained_model)
model = model.merge_and_unload()